In [ ]:
%run 01_upload_file.ipynb

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
def get_player_centers(frame, conf=0.4):
    results = model(frame, conf=conf)
    centers = []

    for r in results:
        for box in r.boxes:
            if int(box.cls[0]) == 0:  # person
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cx = (x1 + x2) // 2
                cy = y2 # Modified for feet
                centers.append((cx, cy))
    return centers


In [ ]:

print("IsOpened:", cap.isOpened())

print("Total frames:", int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("Current frame:", int(cap.get(cv2.CAP_PROP_POS_FRAMES)))

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, frame = cap.read()
assert ret

centers = get_player_centers(frame)
assert len(centers) >= 1, "No players detected"

# Sort by vertical position (top/bottom of frame)
centers = sorted(centers, key=lambda x: x[1], reverse=True)
print(centers)

players = {
    "A": centers[0],
    "B": centers[1] if len(centers) > 1 else None
}

players


In [ ]:
vis = frame.copy()

for pid, (cx, cy) in players.items():
    cv2.circle(vis, (cx, cy), 8, (0,0,255), -1)
    cv2.putText(vis, pid, (cx+5, cy-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)

plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
plt.axis("off")